# Answering Business Questions Using SQL
by Nicholas Archambault

This project will answer questions about the Chinook database, a collection of eleven tables of information on a fictional, digital iTunes-like music store. The data includes information on employees, customers, purchases and product information.

## Exploring the Data

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase

The fictional scenario at hand is as follows: the Chinook record store has just signed a deal with a new record label, and we are tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now -- we know the artist names and the genre of music they produce. The four genres from which to choose are Hip Hop, Punk, Pop, and Blues. 

The goal of this section is to identify which of these genres sell best in the United States, then make a recommendation for which three albums should be purchased of the four options.

Our query yields the number of tracks sold in the United States by genre, in terms of both absolute numbers and percentages of total tracks sold.

In [3]:
%%sql

WITH usa_tracks AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
      g.name genre,
      COUNT(ut.invoice_line_id) tracks_sold,
      ROUND(
          CAST(COUNT(ut.invoice_line_id) AS FLOAT) / (
            SELECT COUNT(*) FROM usa_tracks), 
          4) percentage_sold
FROM  usa_tracks ut
INNER JOIN track t on t.track_id = ut.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5338
Alternative & Punk,130,0.1237
Metal,124,0.118
R&B/Soul,53,0.0504
Blues,36,0.0343
Alternative,35,0.0333
Latin,22,0.0209
Pop,22,0.0209
Hip Hop/Rap,20,0.019
Jazz,14,0.0133


Based on this preliminary analysis, we should purchase the Punk, Blues, and Pop albums. We should keep in mind, though, that theses three genres make up just 17% of total tracks sold. A more promising category might be Rock, which accounts for 53% of total sales.

## Analyzing Employee Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. We can analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

It might prove useful to consider whether any extra columns from the employee table explain any variance that is seen, or whether the variance might instead be indicative of employee performance.

In [4]:
%%sql

WITH emp AS
    (
     SELECT
         i.customer_id,
         c.support_rep_id,
         SUM(i.total) total
     FROM invoice i
     INNER JOIN customer c ON i.customer_id = c.customer_id
     GROUP BY 1
    )

SELECT
    (e.first_name || " " || e.last_name) employee,
    e.hire_date,
    ROUND(SUM(emp.total),2) total_sales
FROM emp
INNER JOIN employee e ON e.employee_id = emp.support_rep_id
GROUP BY 1;

 * sqlite:///chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


Analysis shows that there is a ~20% difference in sales performance between Jane and Steve. This difference, however, corresponds to the difference between their hiring dates, suggesting that quality of the two employees' performances is similar.

## Analyzing Sales by Country

Next, we can examine country-specific metrics, specifically total customers, total sales, average value of sales per customer, and average order value. Countries with only a single customer will be grouped into an 'Other' category.

In [5]:
%%sql

WITH country_data AS (
    SELECT
        c.customer_id,
        i.total,
        i.invoice_id,
        CASE
            WHEN 
                (SELECT COUNT(*) 
                 FROM customer 
                 WHERE country = c.country) = 1
            THEN "Other"
            ELSE c.country
            END
            AS country
    FROM customer c
    INNER JOIN invoice i ON c.customer_id = i.customer_id
    )

SELECT
    country,
    COUNT(DISTINCT(customer_id)) total_customers,
    ROUND(SUM(total), 2) total_sales,
    ROUND(SUM(total)/COUNT(DISTINCT(customer_id)), 2) sales_per_customer,
    ROUND(SUM(total)/COUNT(invoice_id), 2) sales_per_order
FROM 
    (
    SELECT
        cd.*,
        CASE
            WHEN cd.country = "Other" THEN 0
            ELSE 1
            END
            AS sort
    FROM country_data cd
    )
GROUP BY 1
ORDER BY sort DESC, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,sales_per_customer,sales_per_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.57,8.72
Other,15,1094.94,73.0,7.45


We find that the United States leads in total customers and sales. Based on the data, there may be opportunity for  market entry or increased advertising in Czech Republic, United Kingdom and India, all of which feature high values for average sales per customer and average sales per order.

## Albums vs. Tracks

Finally, we can evaluate how individual tracks are purchased. The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do so by choosing each track manually). When customers purchase albums, they are charged the same price as if they had purchased each of those tracks separately.

Management is currently considering changing their purchasing strategy to save money. The strategy under consideration is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We will explore what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

In order to answer the question, we'll have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can then find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

We can use the `EXCEPT` operator wrapped in a `CASE` statement to create a new column that evaluates whether or not each invoice was an album purchase. Summing the binary totals of this column will reveal the desired table of data.

In [6]:
%%sql
WITH first_track_invoice AS (
    SELECT 
        il.invoice_id invoice_id,
        MIN(track_id) first_track
        
    FROM invoice_line il
    GROUP BY 1
    )
SELECT 
    album_purchase,
    COUNT(invoice_id) invoices,
    ROUND(CAST(COUNT(invoice_id) AS FLOAT)/ (SELECT COUNT(*) FROM invoice), 3) pct
    
    FROM (
        SELECT
            fti.*,
            CASE
                WHEN (
                    SELECT t.track_id
                    FROM track t
                    WHERE t.album_id = (SELECT t2.album_id FROM track t2
                                        WHERE t2.track_id = fti.first_track)
                    
                    EXCEPT
                    
                    SELECT il2.track_id FROM invoice_line il2
                    WHERE il2.invoice_id = fti.invoice_id 
                    ) IS NULL
                AND
                    (
                    SELECT il2.track_id FROM invoice_line il2
                    WHERE il2.invoice_id = fti.invoice_id
                    
                    
                    EXCEPT
        
                    SELECT t.track_id FROM track t
                    WHERE t.album_id = (
                        SELECT t2.album_id FROM track t2
                        WHERE t2.track_id = fti.first_track)
                    ) IS NULL
        
            THEN "Yes"
            ELSE "No"
            END
            AS "album_purchase"
        FROM first_track_invoice fti
        )
GROUP BY 1;

 * sqlite:///chinook.db
Done.


album_purchase,invoices,pct
No,500,0.814
Yes,114,0.186


This data reveals that album purchases account for 18.6% of all purchased tracks, nearly a fifth of all revenue. Based on these findings, we can recommend that the company avoid purchasing only select tracks from albums, since that could result in a loss of one fifth of revenue.

## Conclusion

In this project, we have addressed a number of business questions using SQL queries and joins. We reached the following conclusions, to be recommended to the company:
   * The company should purchase Pop, Punk, and Blues albums and be on the lookout for opportunities to purchase more Rock albums, as the Rock genre accounts for over half of all purchases.
   * The company does most of its business in the United States, but new opportunities for growth could be present in Czech Republic, United Kingdom, and India.
   * There is not an appreciable difference in the overall quality of sales employee performance.
   * The company should refrain from changing its purchasing policy to one that buys only certain tracks from albums. Doing so risks losing ~20% of total revenue. 